In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import r2_score
sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

In [98]:
def tag_extraction(
        tag_list,
        ts_end,
        DAYSBACK=61,
        query_digital=False,
):
    TAG_LIST = [x.strip() for x in tag_list]
    TAG_LIST = str(TAG_LIST).replace("[","(").replace("]",")")

    START_DATE = ts_end.split(sep=" ")[0]
    BQ_TABLE_NAME='bq_view_pi_raw_guaiba_regular_refined'

    query = '''
    SELECT
    tag_name,
    tag_value,
    cast(
        concat(tag_date, ' ', tag_tm) as datetime
    ) as tag_datetime

    FROM
    `innovation-datalake-prod.datalake_guaiba_views.@TABLE` WHERE
        tag_status='Good'
        and tag_date >= DATE_ADD(CAST('@TAGDATE' AS DATE), INTERVAL -@DAYSBACK DAY)
        and tag_date <= DATE_ADD(CAST('@TAGDATE' AS DATE), INTERVAL 1 DAY)

        --and tag_date not between '2021-10-17' and '2021-10-24' # parada
        and tag_date not between '2022-01-22' and '2022-02-07' # problema pi
        and tag_date not between '2022-05-23' and '2022-06-06' # parada
        and tag_date not between '2023-05-04' and '2023-05-18' # pg
        and tag_date not between '2023-10-15' and '2023-11-25' # pg bio
        and tag_name IN @TAGLIST
    '''
    query_reg = query.replace(
        '@TABLE', BQ_TABLE_NAME).replace(
            '@TAGLIST', TAG_LIST).replace(
                '@TAGDATE', str(START_DATE)).replace(
                    '@DAYSBACK', str(DAYSBACK)
                )

    job_regular = pd.read_gbq(project_id='innovation-datascience-test', query=query_reg)

    if query_digital:
        BQ_TABLE_NAME='bq_view_pi_raw_guaiba_digital'

        query_dig = query.replace(
        '@TABLE', BQ_TABLE_NAME).replace(
            '@TAGLIST', TAG_LIST).replace(
                '@TAGDATE', str(START_DATE)).replace(
                    '@DAYSBACK', str(DAYSBACK)
                )
        job_digital = pd.read_gbq(project_id='innovation-datascience-test', query=query_dig)
        df = pd.concat(
            [job_regular, job_digital], axis=0
        )

    else:
        df = job_regular.copy()

    df = df.drop_duplicates()

    df_pivot = df.pivot_table(
        columns='tag_name',
        values='tag_value',
        index='tag_datetime',
        aggfunc='first'
        )
    return df_pivot

In [99]:
def tank_level_calculation(
        df,
        level_tag,
        tank_capacity,
        input_flow,
        output_flow,
        data_frequency=12,
        level_follow_over='sensor'
):
    '''
    data_frequency:
        number of records within an hour
    level_follow_over:
        calculated -> to sum over calculated level, otherwise will sum over tag level

    '''
    df = df.sort_index()
    DELTA_M3 = (df[input_flow] - df[output_flow]) / data_frequency
    DELTA_PERC = 100*(DELTA_M3 / tank_capacity)

    if level_follow_over=='calculated':
        DELTA_PERC_CUM = DELTA_PERC.cumsum()
        NIVEL_CALC = DELTA_PERC_CUM + df[level_tag].head(1).values[0]
       # NIVEL_CALC = NIVEL_CALC.shift(-6)
    else:
        NIVEL_CALC_MINUS1 = df[level_tag] + DELTA_PERC
        NIVEL_CALC = NIVEL_CALC_MINUS1.shift(1)

    return np.round(NIVEL_CALC.astype('float'), 2) #np.round(NIVEL_CALC, 2)#, DELTA_M3, DELTA_PERC, DELTA_PERC_CUM, df[level_tag].head(1).values[0]

In [100]:
DATE_RANGE = pd.date_range(start='2023-09-14', periods=10)

In [ ]:
#CBL TANK
TAG_LIST = [
    #INPUT'
    '774F0229.PV',
    #OUTPUT
    '771F5394.M2.PV',
    #TANK LEVEL
    '774L0233.A.PV'
]

INPUT_TAGS = ['774F0229.PV']
OUTPUT_TAGS = ['771F5394.M2.PV']
TAG_NIVEL = '774L0233.A.PV'
CAPACITY = 2400

df = tag_extraction(tag_list=TAG_LIST, ts_end='2023-10-15', DAYSBACK=31)

flows = df[INPUT_TAGS + OUTPUT_TAGS]#.rolling(12).mean()
level = df.loc[flows.index, [TAG_NIVEL]]

df_rolling = pd.concat([flows, level], axis=1)

df_rolling['INPUT'] = df_rolling['774F0229.PV']
df_rolling.dropna(inplace=True)
df_rolling['OUTPUT'] = df_rolling['771F5394.M2.PV'] / 16.67 # coverting lpm in m3ph

df_rolling.index = pd.to_datetime(df_rolling.index)
ncols = 2
nrows = int(np.ceil(len(DATE_RANGE) / ncols))

subdf_lst = []
fig, axes = plt.subplots(nrows, ncols, figsize=(12, nrows*3))
for i, day in enumerate(DATE_RANGE):
    subdf = df_rolling.loc[str(day.date()), : ].between_time('01:00','08:00').resample('1H').mean()
    OUTPT_MEAN = subdf['OUTPUT'].mean()
    INPUT_MEAN = subdf['INPUT'].mean()
    LAST_TANK_LEVEL = subdf['774L0233.A.PV'].iloc[-1]
    DELTA_M3 = (INPUT_MEAN - OUTPT_MEAN)
    DELTA_PERC = 100*(DELTA_M3 / CAPACITY)
    nivel_pred = []
    cont = 0

    while cont < 9:
        LAST_TANK_LEVEL = LAST_TANK_LEVEL + DELTA_PERC
        nivel_pred.append(LAST_TANK_LEVEL)
        cont= cont+1
    
    subdf_2 = df_rolling.loc[str(day.date()), : ].between_time('08:00','16:00').resample('1H').mean()
    subdf_2['nivel_pred'] = nivel_pred
    
    subdf_2[[
        'nivel_pred',
        #'DELTA',
        TAG_NIVEL
            ]].plot(
                #secondary_y=['DELTA'],
                grid=True,
                marker='o',
                ax=fig.axes[i],
                alpha=.7
                )
    subdf_lst.append(subdf_2)
fig.tight_layout()
plt.show();

subdf_comp = pd.concat(subdf_lst)
subdf_comp.dropna(inplace=True)
x = subdf_comp[TAG_NIVEL]
y = subdf_comp['nivel_pred']

R2 = r2_score(x, y)*100
sns.scatterplot(x=x, y=y, alpha=.3)
plt.title(f"R2: {R2:.2f} %")
plt.show();